In [1]:
import pandas as pd

In [2]:
def cuota_parte(cuota,capacidad,SA,PE,SIN,Rea):
    """Función que calcula el contrato de reaguro cuota parte
    
    Inputs:
    Cuota: float
    
    Es el porcentaje de la cuota parte
    
    Capacidad: float
    
    Es la capacidad del contrato
    
    SA: float
    
    Es la suma asegurada
    
    PE: float
    
    Es el valor de la prima emitida
    
    SIN: float
    
     Es el valor del siniestro
     
     Rea:list
     
     Lista con los porcentajes de las aseguradoras
    
    """
    minimo=min(cuota*SA,cuota*capacidad)
    if minimo==cuota*SA:
        Sa=[minimo]
        for ase in Rea:
            Sa+=[(SA-Sa[0])*ase]
        porcentajes=[x/SA for x in Sa]
        primas=[p*PE for p in porcentajes]
        siniestros=[p*SIN for p in porcentajes]
        columnas=["Retención"]+["CP Reaseguradora "+str(r+1) for r  in range(len(Rea))]

    else:
        facultativo=SA-capacidad
        Sa=[capacidad*cuota]
        for ase in Rea:
                Sa+=[capacidad*(1-cuota)*ase]
        Sa+=[facultativo]
        porcentajes=[x/SA for x in Sa]
        primas=[p*PE for p in porcentajes]
        siniestros=[p*SIN for p in porcentajes]
        columnas=["Retención"]+["CP Reaseguradora "+str(r+1) for r  in range(len(Rea))]+['Facultativo']

    df= pd.DataFrame([Sa,primas,siniestros],columns=columnas).T
    df.columns=['Suma Asegurada','Prima','Siniestro']
    return df

In [3]:
#Ejemplo 1
cuota_parte(0.4,2_000_000,600_000,25_000,120_000,[0.3,0.7])

,Suma Asegurada,Prima,Siniestro
Retención,240000.0,10000.0,48000.0
CP Reaseguradora 1,108000.0,4500.0,21600.0
CP Reaseguradora 2,252000.0,10500.0,50400.0


In [4]:
#Ejempolo 2
cuota_parte(0.4,2_000_000,600_000,25_000,120_000,[1,0])

,Suma Asegurada,Prima,Siniestro
Retención,240000.0,10000.0,48000.0
CP Reaseguradora 1,360000.0,15000.0,72000.0
CP Reaseguradora 2,0.0,0.0,0.0


In [5]:
#Ejempolo 3
cuota_parte(0.6,2_000_000,3_500_000,25_000,250_000,[0.4,0.6])

,Suma Asegurada,Prima,Siniestro
Retención,1200000.0,8571.428571,85714.285714
CP Reaseguradora 1,320000.0,2285.714286,22857.142857
CP Reaseguradora 2,480000.0,3428.571429,34285.714286
Facultativo,1500000.0,10714.285714,107142.857143


In [6]:
#Ejemplo 4
cuota_parte(0.4,2_000_000,2_500_000,70_000,500_000,[0.3,0.7])

,Suma Asegurada,Prima,Siniestro
Retención,800000.0,22400.0,160000.0
CP Reaseguradora 1,360000.0,10080.0,72000.0
CP Reaseguradora 2,840000.0,23520.0,168000.0
Facultativo,500000.0,14000.0,100000.0


In [7]:
#Ejemplo 5
cuota_parte(0.3,3_000_000,4_500_000,100_000,800_000,[0.4,0.5,0.9])

,Suma Asegurada,Prima,Siniestro
Retención,900000.0,20000.000000,160000.000000
CP Reaseguradora 1,840000.0,18666.666667,149333.333333
CP Reaseguradora 2,1050000.0,23333.333333,186666.666667
CP Reaseguradora 3,1890000.0,42000.000000,336000.000000
Facultativo,1500000.0,33333.333333,266666.666667
